## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install roboflow pillow matplotlib seaborn tqdm numpy

In [ ]:
# Clone the git repository with the code
import os
if not os.path.exists('Dice-Detection'):
    !git clone https://github.com/Adr44mo/Dice-Detection.git
    %cd Dice-Detection
else:
    %cd Dice-Detection

# Add src to path
import sys
sys.path.append('./src')

## 2. Import Libraries

In [ ]:
import torch
import torch.utils.data as data
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import os

# Import custom modules
from src.dataset import DiceDetectionDataset, collate_fn
from src.model import get_fasterrcnn_model, save_model_checkpoint, get_fasterrcnn_mobilenet
from src.training import train_one_epoch, evaluate, get_optimizer, get_lr_scheduler
from src.metrics import evaluate_map, print_metrics
from src.visualization import (
    plot_class_distribution,
    plot_training_history,
    display_sample_batch,
    visualize_predictions
)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 3. Download Dataset from Roboflow

In [ ]:
from roboflow import Roboflow

# TODO: Add your Roboflow API key
rf = Roboflow(api_key="YOUR_API_KEY")

# Download the dice dataset
project = rf.workspace("workspace-spezm").project("dice-0sexk")
dataset = project.version(2).download("coco")

print(f"Dataset downloaded to: {dataset.location}")

## 4. Explore Dataset

In [ ]:
# Set dataset paths
DATASET_PATH = dataset.location
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
VAL_PATH = os.path.join(DATASET_PATH, "valid")
TEST_PATH = os.path.join(DATASET_PATH, "test")

print(f"Training path: {TRAIN_PATH}")
print(f"Validation path: {VAL_PATH}")
print(f"Test path: {TEST_PATH}")

# Check if test set exists
has_test_set = os.path.exists(TEST_PATH) and os.path.exists(os.path.join(TEST_PATH, "_annotations.coco.json"))
print(f"\nTest set available: {has_test_set}")

# Check annotation file
train_ann_file = os.path.join(TRAIN_PATH, "_annotations.coco.json")
with open(train_ann_file, 'r') as f:
    coco_data = json.load(f)

print(f"\nNumber of training images: {len(coco_data['images'])}")
print(f"Number of annotations: {len(coco_data['annotations'])}")
print(f"Number of categories: {len(coco_data['categories'])}")
print(f"\nCategories: {[cat['name'] for cat in coco_data['categories']]}")

In [ ]:
# Create datasets
train_dataset = DiceDetectionDataset(
    root_dir=TRAIN_PATH,
    annotation_file="_annotations.coco.json",
    split="train"
)

val_dataset = DiceDetectionDataset(
    root_dir=VAL_PATH,
    annotation_file="_annotations.coco.json",
    split="val"
)

# Create test dataset if available
if has_test_set:
    test_dataset = DiceDetectionDataset(
        root_dir=TEST_PATH,
        annotation_file="_annotations.coco.json",
        split="test"
    )
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
else:
    test_dataset = None
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"No test set - will use validation set for final evaluation")

print(f"Number of classes: {train_dataset.num_classes}")

In [ ]:
# Visualize class distribution
train_distribution = train_dataset.get_class_distribution()
print("Training set class distribution:")
for class_name, count in train_distribution.items():
    print(f"  {class_name}: {count}")

plot_class_distribution(train_distribution)

In [ ]:
# Display sample images
display_sample_batch(
    train_dataset,
    num_samples=6,
    class_names=train_dataset.categories
)

## 5. Prepare Data Loaders

In [ ]:
# Hyperparameters
BATCH_SIZE = 4
NUM_WORKERS = 2
NUM_EPOCHS = 10
LEARNING_RATE = 0.005
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(f"Training on: {DEVICE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Number of epochs: {NUM_EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")

In [ ]:
# Create data loaders
train_loader = data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

val_loader = data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

## 6. Initialize Model

In [ ]:
# Create model
model = get_fasterrcnn_mobilenet(
    num_classes=train_dataset.num_classes,
    pretrained=True,
    trainable_backbone_layers=3
)

model.to(DEVICE)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## 7. Setup Optimizer and Scheduler

In [ ]:
# Create optimizer and scheduler
optimizer = get_optimizer(model, lr=LEARNING_RATE)
lr_scheduler = get_lr_scheduler(optimizer, step_size=3, gamma=0.1)

print(f"Optimizer: {optimizer.__class__.__name__}")
print(f"Initial learning rate: {optimizer.param_groups[0]['lr']}")

## 8. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'learning_rate': []
}

best_val_loss = float('inf')
CHECKPOINT_DIR = "checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("Starting training...\n")

In [ ]:
for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    
    # Train
    train_metrics = train_one_epoch(
        model, optimizer, train_loader, DEVICE, epoch + 1
    )
    
    # Evaluate
    val_metrics = evaluate(model, val_loader, DEVICE)
    
    # Update learning rate
    lr_scheduler.step()
    
    # Record history
    history['train_loss'].append(train_metrics['loss'])
    history['val_loss'].append(val_metrics['val_loss'])
    history['learning_rate'].append(optimizer.param_groups[0]['lr'])
    
    # Print summary
    print(f"\nEpoch {epoch + 1} Summary:")
    print(f"  Train Loss: {train_metrics['loss']:.4f}")
    print(f"  Val Loss: {val_metrics['val_loss']:.4f}")
    print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print(f"  Time: {train_metrics['time']:.2f}s")
    
    # Save best model
    if val_metrics['val_loss'] < best_val_loss:
        best_val_loss = val_metrics['val_loss']
        checkpoint_path = os.path.join(CHECKPOINT_DIR, "best_model.pth")
        save_model_checkpoint(
            model, optimizer, epoch + 1, val_metrics['val_loss'],
            checkpoint_path,
            additional_info={'train_loss': train_metrics['loss']}
        )
        print(f"  ✓ New best model saved!")
    
    # Save latest checkpoint
    latest_path = os.path.join(CHECKPOINT_DIR, "latest_model.pth")
    save_model_checkpoint(
        model, optimizer, epoch + 1, val_metrics['val_loss'],
        latest_path
    )

print("\n" + "="*60)
print("Training completed!")
print("="*60)

## 9. Plot Training History

In [ ]:
plot_training_history({
    'Training Loss': history['train_loss'],
    'Validation Loss': history['val_loss'],
    'Learning Rate': history['learning_rate']
})

## 10. Final Evaluation with mAP

Note: Using test set if available, otherwise validation set for final evaluation.

In [ ]:
# Prepare evaluation dataset and loader
# Use test set if available, otherwise use validation set
eval_dataset = test_dataset if has_test_set else val_dataset
eval_loader = data.DataLoader(
    eval_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

print(f"Evaluating on: {'Test' if has_test_set else 'Validation'} set")
print(f"Evaluation dataset size: {len(eval_dataset)}")

# Load best model for evaluation
best_checkpoint = torch.load(os.path.join(CHECKPOINT_DIR, "best_model.pth"))
model.load_state_dict(best_checkpoint['model_state_dict'])
model.to(DEVICE)

print(f"Loaded best model from epoch {best_checkpoint['epoch']}")

In [ ]:
# Evaluate on evaluation set with multiple IoU thresholds
print(f"Evaluating model with mAP metric on {'test' if has_test_set else 'validation'} set...\n")

iou_thresholds = [0.5, 0.75, 0.9, 1.0]
all_map_results = {}

for iou_thresh in iou_thresholds:
    print(f"Computing mAP @ IoU {iou_thresh}...")
    map_results = evaluate_map(
        model, eval_loader, DEVICE,
        iou_threshold=iou_thresh,
        confidence_threshold=0.05
    )
    all_map_results[f'mAP@{iou_thresh}'] = map_results['mAP']
    
    print(f"  mAP@{iou_thresh}: {map_results['mAP']:.4f}")
    
    # Store detailed results for IoU=0.5 (most common)
    if iou_thresh == 0.5:
        detailed_results = map_results

print("\n" + "="*60)
print("Summary of mAP at Different IoU Thresholds:")
print("="*60)
for threshold, mAP_value in all_map_results.items():
    print(f"  {threshold}: {mAP_value:.4f}")
print("="*60)

print("\nDetailed per-class results (IoU=0.5):")
print_metrics(detailed_results, class_names=eval_dataset.categories)

## 11. Visualize Predictions

In [ ]:
# Visualize predictions on random evaluation samples
model.eval()

num_visualizations = 6
indices = np.random.choice(len(eval_dataset), num_visualizations, replace=False)

print(f"Visualizing predictions from {'test' if has_test_set else 'validation'} set:\n")

for idx in indices:
    image, target = eval_dataset[idx]
    
    # Get prediction
    with torch.no_grad():
        prediction = model([image.to(DEVICE)])[0]
    
    # Visualize
    visualize_predictions(
        image,
        prediction,
        class_names=eval_dataset.categories,
        confidence_threshold=0.5
    )

## 12. Save Results

In [ ]:
# Save evaluation results
results_file = "baseline_results.json"

results = {
    'model': 'Faster R-CNN ResNet50-FPN',
    'evaluated_on': 'test' if has_test_set else 'validation',
    'num_epochs': NUM_EPOCHS,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'best_val_loss': float(best_val_loss),
    'final_train_loss': float(history['train_loss'][-1]),
    'mAP@0.5': float(detailed_results['mAP']),
    'mAP_at_iou_thresholds': {k: float(v) for k, v in all_map_results.items()},
    'per_class_ap': {k: float(v) for k, v in detailed_results.items() if k.startswith('AP_class_')},
    'training_history': history
}

with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to {results_file}")
print(f"Note: Model was evaluated on the {'test' if has_test_set else 'validation'} set")
print(f"\nmAP Summary:")
for threshold, mAP_value in all_map_results.items():
    print(f"  {threshold}: {mAP_value:.4f}")

## Summary

This baseline model provides:
- Faster R-CNN with ResNet50-FPN backbone
- Standard training without augmentation
- Evaluation using mAP@0.5
- Per-class performance metrics

Next steps:
1. Implement class-aware sampling in the augmentation notebook
2. Apply mosaic augmentation
3. Compare results with baseline